In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv('covid_19_indonesia_time_series_all.csv')

# Bersihkan kolom
df.columns = df.columns.str.strip().str.lower()
df['date'] = pd.to_datetime(df['date'])

# Cek data awal
df.head()


,date,location iso code,location,new cases,new deaths,new recovered,new active cases,total cases,total deaths,total recovered,...,latitude,new cases per million,total cases per million,new deaths per million,total deaths per million,total deaths per 100rb,case fatality rate,case recovered rate,growth factor of new cases,growth factor of new deaths
0,2020-03-01,ID-JK,DKI Jakarta,2,0,0,2,39,20,75,...,-6.204699,0.18,3.60,0.0,1.84,0.18,51.28%,192.31%,NaN,NaN
1,2020-03-02,ID-JK,DKI Jakarta,2,0,0,2,41,20,75,...,-6.204699,0.18,3.78,0.0,1.84,0.18,48.78%,182.93%,1.0,1.0
2,2020-03-02,IDN,Indonesia,2,0,0,2,2,0,0,...,-0.789275,0.01,0.01,0.0,0.00,0.00,0.00%,0.00%,NaN,NaN
3,2020-03-02,ID-RI,Riau,1,0,0,1,1,0,1,...,0.511648,0.16,0.16,0.0,0.00,0.00,0.00%,100.00%,NaN,NaN
4,2020-03-03,ID-JK,DKI Jakarta,2,0,0,2,43,20,75,...,-6.204699,0.18,3.96,0.0,1.84,0.18,46.51%,174.42%,1.0,1.0


In [2]:
import pandas as pd

# Load data
df = pd.read_csv('covid_19_indonesia_time_series_all.csv')
df.columns = df.columns.str.strip().str.lower()
df['date'] = pd.to_datetime(df['date'])

# Tambahkan total kasus (sudah ada di dataset kamu)
df['year'] = df['date'].dt.year

df.head()


,date,location iso code,location,new cases,new deaths,new recovered,new active cases,total cases,total deaths,total recovered,...,new cases per million,total cases per million,new deaths per million,total deaths per million,total deaths per 100rb,case fatality rate,case recovered rate,growth factor of new cases,growth factor of new deaths,year
0,2020-03-01,ID-JK,DKI Jakarta,2,0,0,2,39,20,75,...,0.18,3.60,0.0,1.84,0.18,51.28%,192.31%,NaN,NaN,2020
1,2020-03-02,ID-JK,DKI Jakarta,2,0,0,2,41,20,75,...,0.18,3.78,0.0,1.84,0.18,48.78%,182.93%,1.0,1.0,2020
2,2020-03-02,IDN,Indonesia,2,0,0,2,2,0,0,...,0.01,0.01,0.0,0.00,0.00,0.00%,0.00%,NaN,NaN,2020
3,2020-03-02,ID-RI,Riau,1,0,0,1,1,0,1,...,0.16,0.16,0.0,0.00,0.00,0.00%,100.00%,NaN,NaN,2020
4,2020-03-03,ID-JK,DKI Jakarta,2,0,0,2,43,20,75,...,0.18,3.96,0.0,1.84,0.18,46.51%,174.42%,1.0,1.0,2020


In [3]:
# Aggregasi per provinsi per tahun
df_agg = df.groupby(['location', 'year']).agg({
    'new cases': 'sum',
    'new deaths': 'sum',
    'new recovered': 'sum',
    'total cases': 'max'
}).reset_index()

# Buat kolom risk_zone
def risk_category(total_cases):
    if total_cases < 10000:
        return 'Rendah'
    elif total_cases < 50000:
        return 'Sedang'
    else:
        return 'Tinggi'

df_agg['risk_zone'] = df_agg['total cases'].apply(risk_category)

# Cek data
df_agg.head()


,location,year,new cases,new deaths,new recovered,total cases,risk_zone
0,Aceh,2020,8746,358,7139,8746,Rendah
1,Aceh,2021,29684,1708,29222,38430,Sedang
2,Aceh,2022,5608,157,5351,44038,Sedang
3,Bali,2020,17593,516,16144,17593,Sedang
4,Bali,2021,96796,3545,94126,114389,Tinggi


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Fitur
X = df_agg[['new cases', 'new deaths', 'new recovered']]
# Target
y = df_agg['risk_zone']

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Fitur
X = df_agg[['new cases', 'new deaths', 'new recovered']]
# Target
y = df_agg['risk_zone']

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Buat model
model = DecisionTreeClassifier(random_state=42)
# Training
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)


In [7]:
# Metrik evaluasi
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Tampilkan
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


Accuracy: 0.95
Precision: 0.89
Recall: 0.96
F1-score: 0.91
